<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/get_data/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [2]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=6meAqiGjSxDOO8-pJxdP15TBPr6pf_edErRFsd0ZcfY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7TcQuOQA9O-RVPi7Pr874fYh65qWcA1mrMFJVsaRQyYB5DC2BIbFY

Successfully saved authorization token.


In [1]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn

In [12]:
continentalUS = ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)

# Import our fire events as features
modis_fires = ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters').filterBounds(continentalUS)
# filter for only fires with greater than 10000 square meters
modis_fires = modis_fires.filter(ee.Filter.gt('area', 10000))
# inspect the first fire event
modis_fires.first().getInfo()

{'geometry': {'coordinates': [[[-64.82080153577421, 43.87890549086832],
    [-64.81640489959548, 43.87890550394663],
    [-64.8164048572393, 43.87451324151839],
    [-64.81201269432611, 43.874513284902505],
    [-64.81201271169233, 43.870116588354],
    [-64.80761596739545, 43.870116602305366],
    [-64.80761595700281, 43.86572438656739],
    [-64.79882709794191, 43.86572438668373],
    [-64.79882709171778, 43.861327698404914],
    [-64.79003818761564, 43.86132771899144],
    [-64.79003817589884, 43.86572438176196],
    [-64.79003824681884, 43.87011656863826],
    [-64.78564602724855, 43.87011658790596],
    [-64.78564599383945, 43.87451329044455],
    [-64.78564594903793, 43.87890549366046],
    [-64.79882712128678, 43.878905498533975],
    [-64.79882712045783, 43.88330214805733],
    [-64.79443487823139, 43.88330216153073],
    [-64.7944349222689, 43.88769435015914],
    [-64.80322376662265, 43.88769437517343],
    [-64.80322380042942, 43.88330218322428],
    [-64.80761601152342, 43.

In [9]:
slo = ee.Geometry.Point([-120.4358, 35.3102])

# Import ImageCollections
landfire_prs = ee.ImageCollection("LANDFIRE/Fire/PRS/v1_2_0").filterBounds(slo)
landfire_evc = ee.ImageCollection("LANDFIRE/Vegetation/EVC/v1_4_0").filterBounds(slo)
# TODO: I don't think that this is the best way to do this, but it works...
prs_palette = landfire_prs.getInfo()['features'][0]['properties']['PRS_class_palette']
evc_palette = landfire_evc.getInfo()['features'][0]['properties']['EVC_class_palette']

# below is useful to inspect the ImageCollection
# landfire_evc.getInfo()

In [13]:
# Set visualization parameters. I'm pretty sure min and max should track to the min and max values of the set at https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Fire_PRS_v1_2_0#bands
prs_vis_params = {
    'min': 1,
    'max': 133,
    'palette': prs_palette
  }
evc_vis_params = {
    'min': 11,
    'max': 172,
    'palette': evc_palette
}

# Create a folium map object.
my_map = folium.Map(location=[35.3102, -120.4358], zoom_start=8)

# Add the fire severity & existing vegetation type images to the map object.
my_map.add_ee_layer(landfire_prs.mosaic(), prs_vis_params, 'Percent Replacement-Severity Fire')
my_map.add_ee_layer(landfire_evc.mosaic(), evc_vis_params, 'Existing Vegetation Cover')
my_map.add_ee_layer(ee.Image().paint(modis_fires), {'palette': '#FF0000'}, 'Fire Events')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)